In [2]:
# Optional: if you haven't installed them yet
# !pip install pandas openpyxl

import pandas as pd
import sqlite3

In [3]:
# Create sample messy data
data = {
    "Name": [" John Doe ", "Jane Smith", " Bob ", "", "Alice"],
    "Email": ["john@example.com", "jane@example.com", "BOB@EXAMPLE.COM", "", None],
    "Age": [30, None, 25, 22, 28],
    "Notes": ["", "Did not disclose age", "  Extra spaces ", "Missing name", ""]
}

df_raw = pd.DataFrame(data)
input_file = "input_data.csv"
df_raw.to_csv(input_file, index=False)
print("✅ Sample CSV created:", input_file)
df_raw

✅ Sample CSV created: input_data.csv


,Name,Email,Age,Notes
0,John Doe,john@example.com,30.0,
1,Jane Smith,jane@example.com,NaN,Did not disclose age
2,Bob,BOB@EXAMPLE.COM,25.0,Extra spaces
3,,,22.0,Missing name
4,Alice,None,28.0,


In [4]:
def transform(df: pd.DataFrame) -> pd.DataFrame:
    print("🔄 Transforming data...")
    
    # Drop rows where ALL columns are null
    df = df.dropna(how='all')
    
    # Normalize text columns (strip + title case)
    text_cols = df.select_dtypes(include='object').columns
    for col in text_cols:
        df[col] = df[col].astype(str).str.strip().str.title()
        df[col] = df[col].replace({'Nan': None, '': None})
    
    # Clean numeric column
    if 'Age' in df.columns:
        df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
    
    print(f"✅ After cleaning: {df.shape[0]} rows, {df.shape[1]} columns")
    return df

df_clean = transform(df_raw.copy())
df_clean

🔄 Transforming data...
✅ After cleaning: 5 rows, 4 columns


,Name,Email,Age,Notes
0,John Doe,John@Example.Com,30.0,None
1,Jane Smith,Jane@Example.Com,NaN,Did Not Disclose Age
2,Bob,Bob@Example.Com,25.0,Extra Spaces
3,None,None,22.0,Missing Name
4,Alice,None,28.0,None


In [5]:
# Save to CSV
csv_output = "cleaned_data.csv"
df_clean.to_csv(csv_output, index=False)
print(f"✅ Saved to {csv_output}")

# Save to SQLite
db_output = "health_data.db"
table_name = "patients"

conn = sqlite3.connect(db_output)
df_clean.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()
print(f"✅ Saved to SQLite table '{table_name}' in {db_output}")

✅ Saved to cleaned_data.csv
✅ Saved to SQLite table 'patients' in health_data.db


In [6]:
# Read back from SQLite to confirm
conn = sqlite3.connect("health_data.db")
df_from_db = pd.read_sql("SELECT * FROM patients", conn)
conn.close()

print("\nData loaded from SQLite:")
df_from_db


Data loaded from SQLite:


,Name,Email,Age,Notes
0,John Doe,John@Example.Com,30.0,None
1,Jane Smith,Jane@Example.Com,NaN,Did Not Disclose Age
2,Bob,Bob@Example.Com,25.0,Extra Spaces
3,None,None,22.0,Missing Name
4,Alice,None,28.0,None


In [7]:
import re

def is_valid_email(email):
    """Simple email validator using regex."""
    if pd.isna(email):
        return False
    email = str(email).strip()
    pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
    return bool(re.match(pattern, email))

def transform(df: pd.DataFrame, required_columns=None) -> pd.DataFrame:
    print("🔄 Transforming data...")
    
    # Ensure we have expected columns (add missing ones as NaN if needed)
    expected_cols = ['Name', 'Email', 'Age', 'Notes']
    for col in expected_cols:
        if col not in df.columns:
            print(f"⚠️ Warning: Column '{col}' missing. Adding as empty.")
            df[col] = None
    
    # Keep only expected columns (optional: ignore extra ones)
    df = df[expected_cols].copy()
    
    # Drop rows where ALL expected fields are null
    df = df.dropna(how='all')
    
    # Clean Name & Notes (text fields)
    for col in ['Name', 'Notes']:
        df[col] = df[col].astype(str).str.strip().str.title()
        df[col] = df[col].replace({'Nan': None, '': None, 'Na': None})
    
    # Clean Email
    df['Email'] = df['Email'].astype(str).str.strip().str.lower()
    df['Email'] = df['Email'].replace({'nan': None, '': None})
    
    # Flag invalid emails as None
    invalid_emails = ~df['Email'].apply(is_valid_email)
    df.loc[invalid_emails, 'Email'] = None
    print(f"❌ Dropped {invalid_emails.sum()} invalid emails")
    
    # Clean Age
    df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
    df = df[df['Age'] >= 0]  # Remove negative ages
    
    print(f"✅ Final dataset: {df.shape[0]} rows, {df.shape[1]} columns")
    return df

# Re-run transformation with enhanced logic
df_clean = transform(df_raw.copy())
df_clean

🔄 Transforming data...
❌ Dropped 2 invalid emails
✅ Final dataset: 4 rows, 4 columns


,Name,Email,Age,Notes
0,John Doe,john@example.com,30.0,None
2,Bob,bob@example.com,25.0,Extra Spaces
3,None,None,22.0,Missing Name
4,Alice,None,28.0,None


In [8]:
# Simulate a CSV with wrong column names
broken_data = pd.DataFrame({
    "Patient Name": ["Alice", "Bob"],
    "Mail": ["alice@email.com", "not-an-email"],
    "Years Old": [30, -5]
})

print("Broken input:")
broken_data

# Run transform — it will auto-fill missing expected cols!
fixed_data = transform(broken_data)
print("\nAfter transform:")
fixed_data

Broken input:
🔄 Transforming data...
⚠️ Warning: Column 'Name' missing. Adding as empty.
⚠️ Warning: Column 'Email' missing. Adding as empty.
⚠️ Warning: Column 'Age' missing. Adding as empty.
⚠️ Warning: Column 'Notes' missing. Adding as empty.
❌ Dropped 0 invalid emails
✅ Final dataset: 0 rows, 4 columns

After transform:


,Name,Email,Age,Notes
